In [1]:
import csv
from copy import deepcopy
from fastai.vision import load_learner, open_image, defaults, Image
from imageio import imwrite
from fastai.core import Path
import torch as torch
from IPython.core.display import HTML
import os
import pandas as pd
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
defaults.device = torch.device('cpu')

In [3]:
def split(img, split_frac):
    img.data[:,:,int(img.data.shape[2]*split_frac)] = 0
    return img

def split_parts(img, split_frac):
    left_im = img.data[:,:,:int(img.data.shape[2]*split_frac)+1]
    right_im = img.data[:,:,int(img.shape[2]*split_frac):]
    return Image(left_im), Image(right_im)

In [7]:
master_path = Path('/Users/nicholasbangs/Notebooks/personal/greek_reader_master/')
multi_path = master_path/'lgi_data'/'gk_letter_imgs'/'multi'
splitter_path =  master_path/'letter_splitter'
class_path = master_path/'models'

In [8]:
splitter_name = 'split_model_2.pkl'
splitter = load_learner(splitter_path, splitter_name)

In [9]:
class_name = 'rn_34.pkl'
class_lrnr = load_learner(class_path, class_name)

In [66]:
split_points = dict()

In [73]:
imgs = iter(multi_path.ls())

In [74]:
# start here

In [75]:
img_name = next(imgs)
#img_name = multi_path/'multi_174.jpg'
im = open_image(img_name)

In [77]:
ip = splitter.predict(im)[1][0][1]
frac = .5 + round(float(ip),2) + .0
new_im = split(deepcopy(im),frac); new_im

In [78]:
i1, i2 = split_parts(open_image(img_name), frac)

In [79]:
split_points[img_name.name] = float(frac)
preds1 = list(zip(class_lrnr.data.classes, class_lrnr.predict(i1)[2]))
preds2 = list(zip(class_lrnr.data.classes, class_lrnr.predict(i2)[2]))
list(zip(sorted(preds1, key=lambda x: x[1], reverse=True)[:3],sorted(preds2, key=lambda x: x[1], reverse=True)[:3]))


[(('α', tensor(1.0000)), ('π', tensor(1.0000))),
 (('multi', tensor(2.5952e-06)), ('hyp', tensor(1.3603e-05))),
 (('ωcap', tensor(7.2878e-07)), ('multi', tensor(9.0489e-06)))]

In [49]:
import config

In [50]:
import file_maintenance

In [57]:
config.letter_dest = splitter_path

In [58]:
file_maintenance.get_letter_dict()

In [65]:
for f in multi_path.ls():
    if '.jpg' in f.name:
        next_id = file_maintenance.get_next_ind('multi')
        os.system('mv {} {}/multi_{}'.format(f, f.parent, next_id))

In [64]:
config.letter_dict

{'multi': (185, array([184])), '.ipynb_checkpoints': (2, array([0, 1]))}

In [48]:
with open(splitter_path/'letter_split_data.csv','a') as f:
    cw = csv.writer(f)a
    for k in split_points:
        cw.writerow([k, split_points[k]])

In [1]:
import os

In [2]:
os.system('mkdir -p greek_reader_master/split_data/auto_tags')

0

In [11]:
df = pd.read_csv('greek_reader_master/letter_splitter/letter_split_data.csv',header=None)

In [13]:
df.columns = ['fname','frac']

In [38]:
from collections import defaultdict

In [47]:
item_count = defaultdict(int)

In [ ]:
for f in multi_path.ls():
    if 'jpg' in f.name:
        frac = df[df['fname']==f.name]['frac'].iloc[0]
        im1, im2 = split_parts(open_image(f), frac)
        im1_label, im2_label = class_lrnr.predict(im1)[0].obj, class_lrnr.predict(im2)[0].obj
        os.system('mkdir -p greek_reader_master/split_data/auto_tags/{}'.format(im1_label))
        os.system('mkdir -p greek_reader_master/split_data/auto_tags/{}'.format(im2_label))
        im1.save('greek_reader_master/split_data/auto_tags/{0}/{0}_{1}.jpg'.format(im1_label, item_count[im1_label]))
        im2.save('greek_reader_master/split_data/auto_tags/{0}/{0}_{1}.jpg'.format(im2_label, item_count[im2_label]))
        item_count[im1_label] += 1
        item_count[im2_label] += 1
        

In [49]:
print('h')

h
